In [8]:
import pandas as pd
import sqlite3
import traceback
db_path = 'mera_parivar.db'

In [9]:
def change_table(connection, table : str, primary_key : str, changes : dict or list):
    """ Given a connection and a table to where to set the changes, first try to update the database.
    If that fails because the ID already exists, then it will insert the values.

    Args:
        connection : _description_
        table : _description_
        changes (dict): {'ID':'value1', 'column2':'value2'}
                (list): [{'ID':'value1', 'column2':'value2'}, {'ID':'value1', 'column2':'value2'}]
    """
    code = 0
    message = ''


    if type(changes) == dict:
        columns = ", ".join([f'"{x}"' for x in changes.keys()])
        excluded_changes = ", ".join([f'"{x}"=excluded."{x}"' for x in changes.keys()])
        values = ", ".join([f'"{x}"' for x in changes.values()])
        query = f"""
        INSERT INTO "{table}"({columns})
        VALUES ({values})
        ON CONFLICT("{primary_key}")
        DO UPDATE SET {excluded_changes}
        """
    elif type(changes) == list:
        columns = ", ".join([f'"{x}"' for x in changes[0].keys()])
        excluded_changes = ", ".join([f'"{x}"=excluded."{x}"' for x in changes[0].keys()])
        values = ", ".join([f'?' for x in changes[0].values()])
        list_of_tuples = [tuple(x.values()) for x in changes]
        query = f"""
        INSERT INTO "{table}"({columns})
        VALUES ({values})
        ON CONFLICT("{primary_key}")
        DO UPDATE SET {excluded_changes}
        """
    try:
        cursor = connection.cursor()
        if type(changes) == dict:
            cursor.execute(query)
            message = f'Se ha actualizado "{primary_key}"="{changes[primary_key]}" correctamente.'
        elif type(changes) == list:
            cursor.executemany(query, list_of_tuples)
            message = f'Se han actualizado {len(changes)} filas correctamente.'
        connection.commit()
    except sqlite3.OperationalError as e:
        code = 1
        message = f'NO se ha actualizado la base de datos debido a OperationalError:\
        \n{traceback.format_exc()}\n\ntratando de ejecutar la query{query}'
    except Exception as e:
        code = 2
        message = f'NO se ha actualizado la base de datos debido a un error no previsto:\
        \n{traceback.format_exc()}\n\ntratando de ejecutar la query{query}'
    return code, message

# Educacion

## Estudiantes

In [10]:
df_students = pd.read_excel('csvs\Educacion.xlsx', sheet_name = 'Hoja1')

In [11]:
df_students.head()

,ID_student,student_name,student_last_name,student_birthdate,student_identification_number,student_sex,student_contact_number,active
0,0,Arjun,Patel,2021-03-15,123456789,M,1234567890,Y
1,1,Arjun,Sharma,2021-07-22,321654987,M,1234567890,Y
2,2,Aarav,Singh,2021-11-09,789456915,M,1234567890,Y
3,3,Dev,Kumar,2021-04-28,123546431,M,1234567890,Y
4,4,Rohan,Gupta,2021-10-03,123654987,M,1234567890,Y


In [14]:
df_students.dtypes

ID_student                                int64
student_name                             object
student_last_name                        object
student_birthdate                datetime64[ns]
student_identification_number             int64
student_sex                              object
student_contact_number                    int64
active                                   object
dtype: object

In [15]:
df_students['student_birthdate'] = df_students['student_birthdate'].astype(str)

In [16]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'e_students', 'ID_student', df_students.to_dict('records'))
con.close()

In [17]:
message

'Se han actualizado 50 filas correctamente.'

In [6]:
dicc_students = df_students.to_dict('records')

[{'ID_student': 0,
  'student_name': 'Arjun',
  'student_last_name': 'Patel',
  'student_birthdate': Timestamp('2021-03-15 00:00:00'),
  'student_identification_number': 123456789,
  'student_sex': 'M',
  'student_contact_number': 1234567890,
  'active': 'Y'},
 {'ID_student': 1,
  'student_name': 'Arjun',
  'student_last_name': 'Sharma',
  'student_birthdate': Timestamp('2021-07-22 00:00:00'),
  'student_identification_number': 321654987,
  'student_sex': 'M',
  'student_contact_number': 1234567890,
  'active': 'Y'},
 {'ID_student': 2,
  'student_name': 'Aarav',
  'student_last_name': 'Singh',
  'student_birthdate': Timestamp('2021-11-09 00:00:00'),
  'student_identification_number': 789456915,
  'student_sex': 'M',
  'student_contact_number': 1234567890,
  'active': 'Y'},
 {'ID_student': 3,
  'student_name': 'Dev',
  'student_last_name': 'Kumar',
  'student_birthdate': Timestamp('2021-04-28 00:00:00'),
  'student_identification_number': 123546431,
  'student_sex': 'M',
  'student_cont

## Cursos

In [18]:
df_cursos = pd.read_excel('csvs\Educacion.xlsx', sheet_name = 'Hoja2')

In [19]:
df_cursos

,ID_course,course_name,course_schedule,course_description
0,NUR,Nursury,morning,NaN
1,LKG,Lower Kinder Garden,morning,NaN
2,UKG,Upper Kinder Garden,morning,NaN
3,WM,Women Program,evening,NaN


In [20]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'e_courses', 'ID_course', df_cursos.to_dict('records'))
con.close()

if code != 0:
    print(message)

## Inscripcion

In [28]:
df_inscrip = pd.read_excel('csvs\Educacion.xlsx', sheet_name = 'Hoja3')

In [29]:
df_inscrip['inscription_date'] = df_inscrip['inscription_date'].astype(str)

In [30]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'e_inscription', 'ID_inscription', df_inscrip.to_dict('records'))
con.close()

if code != 0:
    print(message)

## Prof

In [31]:
df_teach = pd.read_excel('csvs\Educacion.xlsx', sheet_name = 'Hoja4')

In [32]:
df_teach['teacher_birthdate'] = df_teach['teacher_birthdate'].astype(str)

In [33]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'e_teachers', 'ID_teacher', df_teach.to_dict('records'))
con.close()

if code != 0:
    print(message)

## Asignaturas

In [71]:
df_asig = pd.read_excel('csvs\Educacion.xlsx', sheet_name = 'Hoja5')

In [72]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'e_subjects', 'ID_subject', df_asig.to_dict('records'))
con.close()

if code != 0:
    print(message)

## Asistencia

In [36]:
df_asist = pd.read_excel('csvs\Educacion.xlsx', sheet_name = 'Hoja6')

In [38]:
df_asist['attendance_date'] = df_asist['attendance_date'].astype(str)

In [39]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'e_attendance', 'ID_attendance', df_asist.to_dict('records'))
con.close()

if code != 0:
    print(message)

# Notas

In [40]:
df_notas = pd.read_excel('csvs\Educacion.xlsx', sheet_name = 'Hoja7')

In [41]:
df_notas['score_date'] = df_notas['score_date'].astype(str)

In [42]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'e_scores', 'ID_score', df_notas.to_dict('records'))
con.close()

if code != 0:
    print(message)

# Asistencia

## Servicios

In [45]:
df_servicios = pd.read_excel('csvs\Asistencia.xlsx', sheet_name = 'Hoja1')

In [46]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'c_services', 'ID_service', df_servicios.to_dict('records'))
con.close()

if code != 0:
    print(message)

## Solicitantes

In [50]:
df_solic = pd.read_excel('csvs\Asistencia.xlsx', sheet_name = 'Hoja2')

In [51]:
df_solic['applicant_birthdate'] = df_solic['applicant_birthdate'].astype(str)

In [52]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'c_applicants', 'ID_applicant', df_solic.to_dict('records'))
con.close()

if code != 0:
    print(message)

## Tramites

In [53]:
df_tram = pd.read_excel('csvs\Asistencia.xlsx', sheet_name = 'Hoja3')

In [56]:
df_tram['application_date'] = df_tram['application_date'].astype(str)
df_tram['procedure_date'] = df_tram['procedure_date'].astype(str)

In [57]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'c_procedures', 'ID_procedure', df_tram.to_dict('records'))
con.close()

if code != 0:
    print(message)

# Voluntarios

## vol

In [64]:
df_vol = pd.read_excel('csvs\Voluntarios.xlsx', sheet_name = 'Hoja1')

In [65]:
df_vol['volunteer_birthdate'] = df_vol['volunteer_birthdate'].astype(str)

In [66]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'v_volunteers', 'ID_volunteer', df_vol.to_dict('records'))
con.close()

if code != 0:
    print(message)

## tipo vol

In [76]:
df_vol_t = pd.read_excel('csvs\Voluntarios.xlsx', sheet_name = 'Hoja2')

In [77]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'v_volunteer_type', 'ID_volunteer_type', df_vol_t.to_dict('records'))
con.close()

if code != 0:
    print(message)

# Food Bank

## productos

In [79]:
df_fb = pd.read_excel('csvs\Food_Bank.xlsx', sheet_name = 'Hoja1')

In [80]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'b_products', 'ID_product', df_fb.to_dict('records'))
con.close()

if code != 0:
    print(message)

## donaciones

In [81]:
df_don = pd.read_excel('csvs\Food_Bank.xlsx', sheet_name = 'Hoja2')

In [83]:
df_don['expiration_date'] = df_don['expiration_date'].astype(str)

In [84]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'b_donations', 'ID_donation', df_don.to_dict('records'))
con.close()

if code != 0:
    print(message)

## Donantes

In [86]:
df_donor = pd.read_excel('csvs\Food_Bank.xlsx', sheet_name = 'Hoja3')

In [87]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'b_donors', 'ID_donor', df_donor.to_dict('records'))
con.close()

if code != 0:
    print(message)

## Donaciones evento

In [89]:
df_dono_e = pd.read_excel('csvs\Food_Bank.xlsx', sheet_name = 'Hoja4')

In [91]:
df_dono_e['donation_date'] = df_dono_e['donation_date'].astype(str)

In [92]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'b_donation_event', 'ID_donation_event', df_dono_e.to_dict('records'))
con.close()

if code != 0:
    print(message)

## Beneficiarios

In [93]:
df_benef = pd.read_excel('csvs\Food_Bank.xlsx', sheet_name = 'Hoja5')

In [94]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'b_beneficiaries', 'ID_beneficiary', df_benef.to_dict('records'))
con.close()

if code != 0:
    print(message)

## Entregas

In [96]:
df_entreg = pd.read_excel('csvs\Food_Bank.xlsx', sheet_name = 'Hoja6')

In [98]:
df_entreg['delivery_date'] = df_entreg['delivery_date'].astype(str)

In [99]:
con = sqlite3.connect(db_path)
code, message = change_table(con, 'b_deliveries', 'ID_delivery', df_entreg.to_dict('records'))
con.close()

if code != 0:
    print(message)